<a href="https://colab.research.google.com/github/ChavChavC/BT4222/blob/main/FE_NGram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
# Import all the necessary libraries
from datasets import load_dataset

# NLTK modules
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Huggingface Datasets
dataset1 = load_dataset("Jean-Baptiste/financial_news_sentiment")
dataset2 = load_dataset("zeroshot/twitter-financial-news-sentiment")
dataset3 = load_dataset("lukecarlate/general_financial_news")

# Normal Datasets

In [ ]:
dataset1['train']['summary_detail']

In [ ]:
import pandas as pd

df = pd.read_csv('cleaned_data.csv')
df = df.drop(columns=df.columns[0], axis=1)
df[df.isnull().any(axis=1)]
df.dropna(subset=['title'], inplace=True)
data= df['title'].tolist()
labels_df = df[['labels']]
labels_array = df['labels'].tolist()

In [ ]:
import nltk
from nltk.util import ngrams

def extract_ngrams(text, num):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Generate n-grams
    n_grams = ngrams(words, num)

    # Convert n-grams to a list of tuples and return it
    return list(n_grams)


In [ ]:
# ngram as cols in

bigrams_dataset = set()
trigrams_dataset = set()
for i in range(len(data)):
  text = data[i]
  bigrams = extract_ngrams(text, 2)
  trigrams = extract_ngrams(text, 3)

  for bigram in bigrams:
    bigrams_dataset.add(bigram)

  for trigram in trigrams:
    trigrams_dataset.add(trigram)


KeyboardInterrupt: ignored

In [ ]:
print(len(bigrams_dataset))
print(len(trigrams_dataset))

93117
109467


In [ ]:
# tfidf generator
from sklearn.feature_extraction.text import TfidfVectorizer

# For only bigrams
#bigram_vectorizer = TfidfVectorizer(ngram_range=(2, 2))
#bigram_matrix = bigram_vectorizer.fit_transform(data)
#dense_matrix = bigram_matrix.todense()

# Get feature names to see which bigrams are in the vocabulary
#feature_names = bigram_vectorizer.get_feature_names_out()
#bigram_df = pd.DataFrame(bigram_matrix.toarray(), columns=feature_names)

# For both unigrams and bigrams
ngram_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
ngram_matrix = ngram_vectorizer.fit_transform(data)
ngram_dense_matrix = ngram_matrix.todense()

# Get feature names to see which bigrams are in the vocabulary
feature_names = ngram_vectorizer.get_feature_names_out()

  # Display the matrix or feature names
# print(dense_matrix)
#print(len(feature_names))
ngram_df = pd.DataFrame(ngram_matrix.toarray(), columns=feature_names)

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(ngram_df, labels_df,test_size=0.2, random_state=42, shuffle=True)

In [ ]:
ngram_df.columns.values.tolist()[-50:]

['zumiez inc',
 'zumz',
 'zumz ceo',
 'zumz zumiez',
 'zuo',
 'zuo ceo',
 'zuo zuora',
 'zuora',
 'zuora inc',
 'zwei',
 'zwei aromenfirmen',
 'zwillinger',
 'zwillinger calls',
 'zyen',
 'zyen chairman',
 'zyme',
 'zyme hc',
 'zynex',
 'zynex reports',
 'zynga',
 'zynga and',
 'zynga nasdaqznga',
 'zynga shares',
 'zynga top',
 'zynga znga',
 'zyxi',
 'zyxi zynex',
 'závazku',
 'závazku pomáhat',
 'ãƒâ',
 'ãƒâ tablissements',
 'ændring',
 'ændring møns',
 'échelle',
 'échelle nationale',
 'électrobac',
 'électrobac donate',
 'énergir',
 'énergir and',
 'ímpeto',
 'ímpeto nos',
 'öltözik',
 'öltözik kiállítás',
 'österreichische',
 'österreichische post',
 'übernahme',
 'übernahme von',
 'ženám',
 'ženám na',
 '気候の非常事態への対策として大幅な炭素削減を可能にするモバイル技術']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(ngram_matrix, labels_array)  # 'labels' is the array of sentiment labels

importances = clf.feature_importances_

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
# Get indices of sorted importances
sorted_indices = np.argsort(importances)[::-1]

# Select the top N feature indices
N = 1000  # For example, select the top 1000 features
top_feature_indices = sorted_indices[:N]

# Select the top N features from the original feature matrix
X_top_ngrams = ngram_matrix[:, top_feature_indices]

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])